In [ ]:
import pandas as pd
import nltk
import numpy as np
from gensim.models import Word2Vec
from os import listdir
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import gensim
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import *
from keras.layers import *
from keras.callbacks import *
from keras import preprocessing
from keras.preprocessing import sequence
import keras
from sklearn import metrics
import gensim
from keras.utils import np_utils

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
docs = pd.read_csv('/content/drive/MyDrive/data_cleaned_1.csv')
docs = docs.dropna()
docs['dialect_cat'] = docs['dialect'].factorize()[0]
docs['text'] = docs['text'].astype(str)
docs['dialect'] = docs['dialect'].astype(str)
#X_train, X_test, y_train, y_test = train_test_split(docs['text'].values,docs['dialect_cat'].values, test_size=0.25,random_state=1,stratify = docs['dialect_cat'])


In [ ]:
review_text = docs.text.apply(gensim.utils.simple_preprocess)

In [ ]:
review_text

0                                   [بالنهاية, ينتفض, يغير]
1         [يعني, محسوب, البشر, حيونه, ووحشيه, وتطلبون, ا...
2                                      [مبين, كلامه, خليجي]
3                            [يسلملي, مرورك, وروحك, الحلوه]
4                                   [وين, الغيبه, اخ, محمد]
                                ...                        
457996                              [بالعافيه, فقلي, الاكل]
457997                                           [مدح, يبي]
457998    [ايييي, احبها, واحب, طاوله, للكرك, ويفضل, ان, ...
457999               [جيبي, مطرقه, شوفي, اهدم, واهدم, ابوه]
458000    [شنو, ميزان, وشنو, شنووو, المهم, ماعلينا, خلين...
Name: text, Length: 457967, dtype: object

In [ ]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [ ]:
model.build_vocab(review_text, progress_per=1000)

In [ ]:
model.train(review_text, total_examples=model.corpus_count, epochs=model.epochs)

(21941008, 24235990)

In [ ]:
learned_words = list(model.wv.vocab)
learned_words

['بالنهاية',
 'ينتفض',
 'يغير',
 'يعني',
 'محسوب',
 'البشر',
 'حيونه',
 'وتطلبون',
 'الغرب',
 'يحترمكم',
 'ويؤمن',
 'بدينكم',
 'بالإرهاب',
 'مبين',
 'كلامه',
 'خليجي',
 'يسلملي',
 'مرورك',
 'وروحك',
 'الحلوه',
 'وين',
 'الغيبه',
 'اخ',
 'محمد',
 'يااخي',
 'الإرهابي',
 'اذا',
 'عراقي',
 'سعودي',
 'فلسطيني',
 'المشكلة',
 'باب',
 'وماعنده',
 'رحمه',
 'المساس',
 'الله',
 'يرضى',
 'خلصت',
 'مشاكل',
 'العرب',
 'واختلفنا',
 'ابو',
 'الطفل',
 'المغردين',
 'سألوا',
 'الكارثة',
 'اللي',
 'وصلتك',
 'مطلبي',
 'يقدم',
 'استقالته',
 'وفوگاها',
 'اعتذار',
 'خلص',
 'والله',
 'لعيونكم',
 'انا',
 'ماعندي',
 'شيء',
 'بالعكس',
 'متابعيني',
 'بعتز',
 'صافية',
 'لبن',
 'يمكن',
 'سؤال',
 'فات',
 'الكثير',
 'يصور',
 'شنو',
 'موقفه',
 'يوثق',
 'بطوله',
 'المفروض',
 'المصور',
 'يحال',
 'للقضاء',
 'لان',
 'نظرات',
 'الطفله',
 'تستنجد',
 'باتجاه',
 'الكاميرا',
 'اولا',
 'اني',
 'ردت',
 'رجل',
 'واللي',
 'ذكر',
 'حجابها',
 'ثانيا',
 'انت',
 'شدعوه',
 'صايره',
 'محامي',
 'صحيح',
 'مقولة',
 'العراق',
 'بلد',
 'الغرا

In [ ]:
model.wv.most_similar('مصر')

[('المصريين', 0.831017255783081),
 ('تونس', 0.8219223022460938),
 ('ليبيا', 0.8008831143379211),
 ('السودان', 0.7911624908447266),
 ('دول', 0.7898327708244324),
 ('السعودية', 0.7621197700500488),
 ('الجزائر', 0.7568091750144958),
 ('الاخوان', 0.7526887655258179),
 ('نكسجية', 0.7330007553100586),
 ('السيسي', 0.7294926643371582)]

In [ ]:
w2v = dict(zip(model.wv.index2word, model.wv.syn0))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


In [ ]:
tok = Tokenizer()
tok.fit_on_texts(docs['text'])
vocab_size = len(tok.word_index) + 1
encd_rev = tok.texts_to_sequences(docs['text'])

In [ ]:
max_rev_len=150  # max lenght of a review
vocab_size = len(tok.word_index) + 1  # total no of words
embed_dim=100 # embedding dimension as choosen in word2vec constructor

In [ ]:
pad_rev= pad_sequences(encd_rev, maxlen=max_rev_len, padding='post')
pad_rev.shape 

(457967, 150)

In [ ]:
embed_matrix=np.zeros(shape=(vocab_size,embed_dim))
for word,i in tok.word_index.items():
  embed_vector=w2v.get(word)
  if embed_vector is not None:  # word is in the vocabulary learned by the w2v model
    embed_matrix[i]=embed_vector
  # if word is not found then embed_vector corressponding to that vector will stay zero.

In [ ]:
print(embed_matrix[14])

[-2.36224318 -0.14007881  0.95432329  1.11853325 -2.80964637  0.04868238
  2.40537667  0.74275243 -1.246315    0.81614244  0.18675432  1.06637359
  4.68978643  0.78807408 -0.14891374 -0.64276379  5.38169956 -0.25556257
 -1.35947752 -3.3997314  -1.17615819  1.07217145  2.25972748  0.55375522
 -3.56282854 -1.28496182 -1.0118959  -2.62956524 -1.10496879  0.901335
 -0.6554178  -1.0120455  -1.98248076 -1.04443014  3.57782769 -0.44859731
  0.74742866  0.52487385  0.27980724  3.52815151 -1.07006311  4.51724815
  0.51498705  0.36560112 -0.16472706 -1.65774369 -1.81995881 -1.04331732
  1.90272498  3.44427681 -1.5019201   1.48719907  1.64984727  0.19379373
  1.46014619  4.84286547  0.02903775 -0.90826356 -1.15646887  1.95957625
 -0.77025723 -1.31562757  0.94259489 -2.65476918 -0.54953468  1.63329279
  4.21494102 -0.51576746 -1.64357376 -0.33046603  0.89141369  1.68479264
 -1.0021888   0.055743    3.2873826  -1.8529532   0.59190744 -0.49862906
  3.54263592 -1.19309032 -3.20725369  0.36701366  0.8

In [ ]:
Y=keras.utils.np_utils.to_categorical(docs['dialect_cat'])  # one hot target as required by NN.
x_train,x_test,y_train,y_test=train_test_split(pad_rev,Y,test_size=0.20,random_state=42)

In [ ]:
from keras.initializers import Constant
from keras.layers import ReLU
from keras.layers import Dropout
model=Sequential()
model.add(Embedding(input_dim=vocab_size,output_dim=embed_dim,input_length=max_rev_len,embeddings_initializer=Constant(embed_matrix)))
model.add(Conv1D(32, 3, padding='same', activation='relu'))
model.add(Dropout(0.2))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(18, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 150, 100)          52167000  
                                                                 
 conv1d (Conv1D)             (None, 150, 32)           9632      
                                                                 
 dropout (Dropout)           (None, 150, 32)           0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 75, 32)           0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 2400)              0         
                                                                 
 dense (Dense)               (None, 250)               600250    
                                                        

In [ ]:
model.fit(x_train,y_train,epochs=5,batch_size=64,validation_data=(x_test,y_test))

Epoch 1/5
5725/5725 [==============================] - 300s 51ms/step - loss: 2.0451 - accuracy: 0.3454 - val_loss: 1.7308 - val_accuracy: 0.4484
Epoch 2/5
5725/5725 [==============================] - 288s 50ms/step - loss: 1.4569 - accuracy: 0.5326 - val_loss: 1.6621 - val_accuracy: 0.4840
Epoch 3/5
5725/5725 [==============================] - 290s 51ms/step - loss: 1.0243 - accuracy: 0.6758 - val_loss: 1.8204 - val_accuracy: 0.4763
Epoch 4/5
5725/5725 [==============================] - 290s 51ms/step - loss: 0.7617 - accuracy: 0.7615 - val_loss: 2.1142 - val_accuracy: 0.4642
Epoch 5/5
5725/5725 [==============================] - 291s 51ms/step - loss: 0.6123 - accuracy: 0.8081 - val_loss: 2.4361 - val_accuracy: 0.4584
